In [1]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

# load model
from pathlib import Path
from cobra.io import read_sbml_model
mitocore = read_sbml_model(Path().cwd() /"models"/  "mitocore_12918_2017_500_MOESM4_ESM.xml") 
mod = read_sbml_model(Path().cwd() / 'cs_mods' / 'Human-GEM.xml' )

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


In [2]:
mitocore.objective = 'OF_ATP_MitoCore'

In [3]:
essential_rxns = cobra.flux_analysis.find_essential_reactions(mitocore)
essential_rxns

{<Reaction Biomass_MitoCore at 0x1d415966fb0>,
 <Reaction EX_biomass_e at 0x1d415552bf0>,
 <Reaction OF_ATP_MitoCore at 0x1d415582d70>}

In [4]:
from cobra.flux_analysis import single_reaction_deletion
rxn_kos = single_reaction_deletion(mitocore, mitocore.reactions[:])
rxn_kos['ids'] = sum([list((rxn)) for rxn in rxn_kos['ids']], [])

In [5]:
from src.get_info import make_rxn_df
import pandas as pd
from cobra.flux_analysis import flux_variability_analysis
fba = mitocore.optimize().to_frame()
fva = flux_variability_analysis(mitocore, loopless=False)
pfba = cobra.flux_analysis.pfba(mitocore).to_frame()
pfba.columns = pfba.columns+'_pFBA'
df = make_rxn_df(mitocore)
df = pd.concat([df,fva,fba,pfba],axis=1)
df = df.merge(rxn_kos, left_on='rxn_ID', right_on='ids')
df

,rxn_ID,cell,lb,ub,sbo,name,subsystem,reaction,met_IDs,GPR,minimum,maximum,fluxes,reduced_costs,fluxes_pFBA,reduced_costs_pFBA,ids,growth,status
0,EX_2hb_e,None,-1000.0,1000.0,SBO:0000627,EX_2hb_e,,2-Hydroxybutanoic acid[External] <=>,2hb_e <=>,,0.000,0.000,0.000,-0.0,0.000,2.0,EX_2hb_e,100.892304,optimal
1,EX_ac_e,None,-1000.0,1000.0,SBO:0000627,EX_ac_e,,Acetate[External] <=>,ac_e <=>,,0.000,0.000,0.000,-0.0,0.000,2.0,EX_ac_e,100.892304,optimal
2,EX_acac_e,None,-1000.0,1000.0,SBO:0000627,EX_acac_e,,Acetoacetate[External] <=>,acac_e <=>,,-0.114,-0.114,-0.114,-0.0,-0.114,2.0,EX_acac_e,100.805015,optimal
3,EX_akg_e,None,-1000.0,1000.0,SBO:0000627,EX_akg_e,,2-Oxoglutarate[External] <=>,akg_e <=>,,0.000,0.000,0.000,-0.0,0.000,2.0,EX_akg_e,100.892304,optimal
4,EX_ala_B_e,None,-1000.0,1000.0,SBO:0000627,EX_ala_B_e,,beta-Alanine[External] <=>,ala_B_e <=>,,0.000,0.000,0.000,-0.0,0.000,2.0,EX_ala_B_e,100.892304,optimal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,COt,None,-1000.0,1000.0,,R_COt CO transporter via diffusion,Boundary Conditions - Misc,CO[Cytosol] <=> CO[External],co_c <=> co_e,,0.000,0.000,0.000,-0.0,0.000,2.0,COt,100.892304,optimal
551,NOt,None,-1000.0,1000.0,,R_NOt NO transport (diffusion),Boundary Conditions - Misc,Nitric oxide[External] <=> Nitric oxide[Cytosol],no_e <=> no_c,,0.000,0.000,0.000,-0.0,0.000,2.0,NOt,100.892304,optimal
552,PCHOLHSTDe,None,-1000.0,1000.0,,R_PCHOLHSTDe Phosphatidylcholine,Boundary Conditions - Lipids,Phosphatidylcholine[Cytosol] <=> Phosphatidylc...,pchol_hs_c <=> pchol_hs_e,,0.000,0.000,0.000,-0.0,0.000,2.0,PCHOLHSTDe,100.892304,optimal
553,PSt3,None,-1000.0,1000.0,,R_PSt3 Phosphatidylserine,Boundary Conditions - Lipids,Phosphatidylserine[External] <=> Phosphatidyls...,ps_hs_e <=> ps_hs_c,,0.000,0.000,0.000,-0.0,0.000,2.0,PSt3,100.892304,optimal


In [6]:
# _MitoCore reactions
l = [r.id for r in mitocore.reactions if '_MitoCore' in r.id]
df[df['rxn_ID'].isin(l)]

,rxn_ID,cell,lb,ub,sbo,name,subsystem,reaction,met_IDs,GPR,minimum,maximum,fluxes,reduced_costs,fluxes_pFBA,reduced_costs_pFBA,ids,growth,status
70,OF_ATP_MitoCore,None,0.0,1000.0,,R_OF_ATP_MitoCore Maximum ATP production (ATP ...,Objective Function - ATP,ATP[Cytosol] + H2O[Cytosol] --> ADP[Cytosol] +...,atp_c + h2o_c --> adp_c + biomass_c + h_c + pi_c,,100.892304,100.892304,100.892304,0.000000e+00,100.892304,-2.000000,OF_ATP_MitoCore,0.000000,optimal
71,OF_HEME_MitoCore,None,0.0,1000.0,,R_OF_HEME_MitoCore Maximum Heme production,Objective Function - Heme,Heme[Mitochondrion] --> Biomass[Mitochondrion],pheme_m --> biomass_m,,0.000000,0.000000,0.000000,0.000000e+00,0.000000,-2.000000,OF_HEME_MitoCore,100.892304,optimal
72,OF_LIPID_MitoCore,None,0.0,1000.0,,R_OF_LIPID_MitoCore Maximum lipid synthesis (M...,Objective Function - Lipid synthesis,0.18 Cardiolipin[Mitochondrion] + 0.4 Phosphat...,0.18 clpn_hs_m + 0.4 pchol_hs_m + 0.34 pe_hs_m...,,0.000000,0.000000,0.000000,0.000000e+00,0.000000,-2.000000,OF_LIPID_MitoCore,100.892304,optimal
73,OF_PROTEIN_MitoCore,None,0.0,1000.0,,R_OF_PROTEIN_MitoCore Maxmium production of am...,Objective Function - Protein Synthesis,0.76 L-Alanine[Mitochondrion] + 0.54 L-Arginin...,0.76 ala_L_m + 0.54 arg_L_m + 0.42 asn_L_m + 0...,,0.000000,0.000000,0.000000,-2.553646e+01,0.000000,2747.339569,OF_PROTEIN_MitoCore,100.892304,optimal
107,CI_MitoCore,None,-1000.0,1000.0,,R_CI_MitoCore NADH dehydrogenase,Electron transport chain CI,3.996 Proton Motive Force[Mitochondrion] + H[M...,3.996 PMF_m + h_m + nadh_m + 0.002 o2_m + 0.99...,ENSG00000115286 and ENSG00000110717 and ENSG00...,29.368523,29.368523,29.368523,-1.942890e-16,29.368523,-2.000000,CI_MitoCore,69.895555,optimal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,FOLt_MitoCore,None,0.0,1000.0,,R_FOLt_MitoCore Folate,Boundary Conditions - Misc,Folate[External] --> Folate[Cytosol],fol_e --> fol_c,,0.000000,0.000000,0.000000,0.000000e+00,0.000000,-2.000000,FOLt_MitoCore,100.892304,optimal
546,NADHt_MitoCore,None,0.0,0.0,,R_NADHt_MitoCore NADH pool,Boundary Conditions - Misc,NADH[External] --> NADH[Cytosol],nadh_e --> nadh_c,,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2.000000,NADHt_MitoCore,100.892304,optimal
547,NADt_MitoCore,None,0.0,0.0,,R_NADt_MitoCore NAD+ pool,Boundary Conditions - Misc,NAD[External] --> NAD[Cytosol],nad_e --> nad_c,,0.000000,0.000000,0.000000,5.871919e-03,0.000000,-17.203487,NADt_MitoCore,100.892304,optimal
548,NADHtm_MitoCore,None,0.0,0.0,,R_NADHtm_MitoCore NADH pool mitochondrial,Boundary Conditions - Misc,NADH[External] --> NADH[Mitochondrion],nadh_e --> nadh_m,,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2.000000,NADHtm_MitoCore,100.892304,optimal


In [7]:
# version of mitocore without biomass metabolites (no_bm)
mitocore_no_bm = mitocore.copy()
bm_rxns = [m for m in mitocore_no_bm.metabolites if 'biomass' in m.id]
mitocore_no_bm.remove_metabolites(bm_rxns)

Read LP format model from file C:\Users\prins\AppData\Local\Temp\tmp4sjg86hk.lp
Reading time = 0.00 seconds
: 441 rows, 1110 columns, 4340 nonzeros


In [8]:
mitocore.reactions.get_by_id('OF_ATP_MitoCore')

Reaction identifier,OF_ATP_MitoCore
Name,R_OF_ATP_MitoCore Maximum ATP production (ATP hydrolysis)
Memory address,0x1d415582d70
Stoichiometry,atp_c + h2o_c --> adp_c + biomass_c + h_c + pi_c ATP + H2O --> ADP + Biomass + H + Orthophosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [9]:
mitocore_no_bm.reactions.get_by_id('OF_ATP_MitoCore')

Reaction identifier,OF_ATP_MitoCore
Name,R_OF_ATP_MitoCore Maximum ATP production (ATP hydrolysis)
Memory address,0x1d460a94c10
Stoichiometry,atp_c + h2o_c --> adp_c + h_c + pi_c ATP + H2O --> ADP + H + Orthophosphate
GPR,
Lower bound,0.0
Upper bound,1000.0


In [15]:
# add no_bm version of OF_ATP rxn to mitocore model
OF_ATP_MitoCore_no_bm = mitocore_no_bm.reactions.get_by_id('OF_ATP_MitoCore').copy()
OF_ATP_MitoCore_no_bm.id = OF_ATP_MitoCore_no_bm.id + '_no_bm'
mitocore.add_reactions([OF_ATP_MitoCore_no_bm])

In [49]:
from src.analysis import create_permutation_dicts
mitocore_dict = {'EX_o2_e':[(-10,1000),(-1000,1000)],'EX_glc_D_e':[(-1,1000),(-30,1000)]} # allow oxygen and glucose in
mitocore_dict_p = create_permutation_dicts(mitocore_dict)

In [52]:
from src.modify_model import close_EX    
with mitocore_no_bm as model:
    #close_EX(model)
    model.objective = 'OF_ATP_MitoCore'    
    for d in mitocore_dict_p:
        for k in d:
            model.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict
        print(model.id)
        print(mitocore.objective.expression)
        print(d)
        fba = model.optimize()        
        print(fba.objective_value)
        fba_results = fba.to_frame() # df FBA results
        fba_results.columns = fba_results.columns + '_' + str(d) # add RPE_ATP to FBA column names

S1SBMLmodel
1.0*OF_ATP_MitoCore - 1.0*OF_ATP_MitoCore_reverse_653d9
{'EX_o2_e': (-10, 1000), 'EX_glc_D_e': (-1, 1000)}
53.264923496949606
S1SBMLmodel
1.0*OF_ATP_MitoCore - 1.0*OF_ATP_MitoCore_reverse_653d9
{'EX_o2_e': (-10, 1000), 'EX_glc_D_e': (-30, 1000)}
53.264923496949606
S1SBMLmodel
1.0*OF_ATP_MitoCore - 1.0*OF_ATP_MitoCore_reverse_653d9
{'EX_o2_e': (-1000, 1000), 'EX_glc_D_e': (-1, 1000)}
100.89230387215602
S1SBMLmodel
1.0*OF_ATP_MitoCore - 1.0*OF_ATP_MitoCore_reverse_653d9
{'EX_o2_e': (-1000, 1000), 'EX_glc_D_e': (-30, 1000)}
100.89230387215602


In [51]:
from src.modify_model import close_EX    
with mitocore as model:
    #close_EX(model)
    model.objective = 'OF_ATP_MitoCore_no_bm'    
    for d in mitocore_dict_p:
        for k in d:
            model.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict
        print(model.id)
        print(mitocore.objective.expression)
        print(d)
        fba = model.optimize()        
        print(fba.objective_value)
        fba_results = fba.to_frame() # df FBA results
        fba_results.columns = fba_results.columns + '_' + str(d) # add RPE_ATP to FBA column names

S1SBMLmodel
1.0*OF_ATP_MitoCore_no_bm - 1.0*OF_ATP_MitoCore_no_bm_reverse_733d5
{'EX_o2_e': (-10, 1000), 'EX_glc_D_e': (-1, 1000)}
53.26492349694949
S1SBMLmodel
1.0*OF_ATP_MitoCore_no_bm - 1.0*OF_ATP_MitoCore_no_bm_reverse_733d5
{'EX_o2_e': (-10, 1000), 'EX_glc_D_e': (-30, 1000)}
53.264923496949606
S1SBMLmodel
1.0*OF_ATP_MitoCore_no_bm - 1.0*OF_ATP_MitoCore_no_bm_reverse_733d5
{'EX_o2_e': (-1000, 1000), 'EX_glc_D_e': (-1, 1000)}
100.89230387215603
S1SBMLmodel
1.0*OF_ATP_MitoCore_no_bm - 1.0*OF_ATP_MitoCore_no_bm_reverse_733d5
{'EX_o2_e': (-1000, 1000), 'EX_glc_D_e': (-30, 1000)}
100.89230387215603


In [47]:
from src.modify_model import close_EX    
with mitocore_no_bm as model:
    close_EX(model)
    model.objective = 'OF_ATP_MitoCore'    
    for d in mitocore_dict_p:
        for k in d:
            model.reactions.get_by_id(k).bounds = d[k] # set bounds defined in dict
        print(model.id)
        print(mitocore.objective.expression)
        print(d)
        fba = model.optimize()        
        print(fba.objective_value)
        fba_results = fba.to_frame() # df FBA results
        fba_results.columns = fba_results.columns + '_' + str(d) # add RPE_ATP to FBA column names

S1SBMLmodel
1.0*OF_ATP_MitoCore - 1.0*OF_ATP_MitoCore_reverse_653d9
{'EX_o2_e': (-10, 1000), 'EX_glc_D_e': (-1, 1000)}
0.0
S1SBMLmodel
1.0*OF_ATP_MitoCore - 1.0*OF_ATP_MitoCore_reverse_653d9
{'EX_o2_e': (-10, 1000), 'EX_glc_D_e': (-300, 1000)}
0.0
S1SBMLmodel
1.0*OF_ATP_MitoCore - 1.0*OF_ATP_MitoCore_reverse_653d9
{'EX_o2_e': (-1000, 1000), 'EX_glc_D_e': (-1, 1000)}
0.0
S1SBMLmodel
1.0*OF_ATP_MitoCore - 1.0*OF_ATP_MitoCore_reverse_653d9
{'EX_o2_e': (-1000, 1000), 'EX_glc_D_e': (-300, 1000)}
0.0
